In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
import xgboost as xgb
from catboost import CatBoostRegressor
%matplotlib inline

In [2]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")
weather = pd.read_csv("weather.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
train_df = train_df.drop(columns=['amount'])

In [4]:
def Data_cre(df):
    # --- area を分解して複数行に展開 ---
    df["area_list"] = df["area"].str.split("_")

    # explodeで行を増やす
    df = df.explode("area_list")
    df = df.drop(columns=["area"])
    df = df.rename(columns={"area_list": "area"})

    # --- 日付フォーマット揃え ---
    df["date"] = df["date"].astype(str)
    df["date"] = pd.to_datetime(df["date"])

    # --- weather と結合 ---
    df = pd.merge(df, weather, on=["date", "area"], how="left")

    # --- 年月日列を追加 ---
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day

    # --- max_temp_time / min_temp_time 分解 ---
    df["max_temp_time"] = pd.to_datetime(df["max_temp_time"])
    df["maxtemp_year"] = df["max_temp_time"].dt.year
    df["maxtemp_month"] = df["max_temp_time"].dt.month
    df["maxtemp_day"] = df["max_temp_time"].dt.day
    df["maxtemp_hour"] = df["max_temp_time"].dt.hour

    df["min_temp_time"] = pd.to_datetime(df["min_temp_time"])
    df["mintemp_year"] = df["min_temp_time"].dt.year
    df["mintemp_month"] = df["min_temp_time"].dt.month
    df["mintemp_day"] = df["min_temp_time"].dt.day
    df["mintemp_hour"]= df["min_temp_time"].dt.hour

    # --- 不要列削除 ---
    df = df.drop(columns=["date", "max_temp_time", "min_temp_time"])

    # --- category 型に変換 ---
    df["kind"] = df["kind"].astype("category")
    df["area"] = df["area"].astype("category")
    return df
weather["date"] = pd.to_datetime(weather["date"])

train_df = Data_cre(train_df)
test_df = Data_cre(test_df) 

In [5]:
### chatgpt
from sklearn.model_selection import KFold
import numpy as np
import lightgbm as lgb

models = {}  # kind → model list or averaged は辞書に保存

categorical_cols = ['kind', 'area'] 
K = 5  # ← Fold数

for kd, group in train_df.groupby("kind", observed=False):

    X = group.drop(columns=["mode_price"],axis=1)
    y = group["mode_price"]

    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    fold_models = []  # foldごとのモデルを保存

    for fold, (train_index, valid_index) in enumerate(kf.split(X)):

        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        train_data = lgb.Dataset(X_train, y_train)
        valid_data = lgb.Dataset(X_valid, y_valid)

        model = CatBoostRegressor(
            iterations=20,
            learning_rate=0.1,
            depth=6,
            od_type='Iter',           # 早期停止タイプ
            od_wait=5,               # 改善が無い場合50イテレーション後に停止
            use_best_model=True,      # ベストモデルを選択
            eval_metric='RMSE',        # 評価指標
            random_state=fold,
            loss_function='RMSE',
            logging_level='Silent'
        )

        model.fit(
            X_train, y_train,
            cat_features=categorical_cols,
            eval_set=[(X_valid, y_valid)],
            verbose=True
        )
        
        fold_models.append(model)

    models[kd] = fold_models  # kindごとにfoldモデルを保存

In [6]:
import numpy as np

pred_list = []

for kd, group in test_df.groupby("kind"):

    fold_models = models[kd]  # ← foldモデルが複数入っている
    X_test = group.copy()

    # 全 fold モデルの予測を行い平均
    fold_pred = np.mean([m.predict(X_test) for m in fold_models], axis=0)

    X_test["mode_price"] = fold_pred
    pred_list.append(X_test)

# まとめ・並び順合わせ
result = pd.concat(pred_list).sort_index()

# 日付整形（year,month,day → YYYYMMDD）
result["date"] = (
    result["year"].astype(str)
    + result["month"].astype(str).str.zfill(2)
    + result["day"].astype(str).str.zfill(2)
)
result["date"] = result["date"].astype(int)

# 提出形式に整形
submit = result[["kind", "date", "mode_price"]]

submit

C:\Users\ND013\AppData\Local\Temp\ipykernel_25156\846674396.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for kd, group in test_df.groupby("kind"):


,kind,date,mode_price
0,だいこん,20220502,994.007843
1,だいこん,20220502,994.190282
2,だいこん,20220502,994.190282
3,だいこん,20220506,993.375845
4,だいこん,20220506,993.558284
...,...,...,...
815,ミニトマト,20220528,115.416780
816,ミニトマト,20220530,115.648351
817,ミニトマト,20220530,115.648351
818,ミニトマト,20220531,115.740009


In [7]:
submit_mean = (
    submit.groupby(["kind", "date"], as_index=False)["mode_price"]
          .mean()
)
# ① area 側の kind の出現順を取得
kind_order = test_df["kind"].drop_duplicates().tolist()

# ② price 側の kind を Categorical に変換して順序を固定
submit_mean["kind"] = pd.Categorical(submit_mean["kind"], categories=kind_order, ordered=True)

# ③ 並び替え
df_price_sorted = submit_mean.sort_values(["kind", "date"]).reset_index(drop=True)

C:\Users\ND013\AppData\Local\Temp\ipykernel_25156\975664683.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  submit.groupby(["kind", "date"], as_index=False)["mode_price"]


In [8]:
df_price_sorted.to_csv("submit_cat.csv", index=False)
df_price_sorted

,kind,date,mode_price
0,だいこん,20220502,994.129469
1,だいこん,20220506,993.497471
2,だいこん,20220507,993.720465
3,だいこん,20220509,993.581319
4,だいこん,20220510,992.041979
...,...,...,...
315,ミニトマト,20220526,115.312556
316,ミニトマト,20220527,115.351913
317,ミニトマト,20220528,115.416780
318,ミニトマト,20220530,115.648351
